In [12]:
import warnings
warnings.filterwarnings("ignore")  # "error", "ignore", "always", "default", "module" or "once"

In [13]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

from federated_utils_fedavg_copy import *



In [14]:
#declear path to your data
drebin_data_path = r'H:\GIT project\DW-FedAvg\data\drebin.csv'
malgenome_data_path = r'H:\GIT project\DW-FedAvg\data\malgenome.csv'
kronodroid_data_path = r'H:\GIT project\DW-FedAvg\data\kronodroid.csv'
TUANDROMD_data_path=r'H:\GIT project\DW-FedAvg\data\TUANDROMD.csv'



Drebin_data = pd.read_csv(drebin_data_path, header = None)

Malgenome_data = pd.read_csv(malgenome_data_path)

Tuandromd_data=pd.read_csv(TUANDROMD_data_path)

kronodroid_data=pd.read_csv(kronodroid_data_path)
Kronodroid_data = kronodroid_data.iloc[:,range(1,kronodroid_data.shape[1])]

In [18]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import random



def fgsm_attack(model, X, y, epsilon,loss, optimizer):
    # Generate adversarial examples
    model.eval()
    X_adv = X.detach().clone()
    X_adv.requires_grad = True
    outputs = model(X_adv)
    loss = loss(outputs, y)
    # print("adver" , loss)
    loss.backward()
    grad_sign = X_adv.grad.data.sign()
    X_adv = X_adv + epsilon * grad_sign
    X_adv = torch.clamp(X_adv, 0, 1)
    train_loader = DataLoader(TensorDataset(torch.tensor(X_adv, dtype=torch.float32),
                                        torch.tensor(y, dtype=torch.float32)),
                          batch_size=32, shuffle=True)
    return train_loader

def train_model(model, train_loader, loss_fn, optimizer):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        # print("adver" , loss)

        loss.backward()
        optimizer.step()

all_avg = []
all_std = []

n_clients = [5, 10, 15]
n_round = [200, 5000]

dataset = ['Drebin', 'Malgenome', 'Kronodroid', 'Tuandromd']

# for d in range(0,len(dataset)):
for d in range(0, 2):
    if d == 0:
        use_data = Drebin_data
    elif d == 1:
        use_data = Malgenome_data
    elif d == 2:
        use_data = Kronodroid_data
    elif d == 3:
        use_data = Tuandromd_data

    print('===================================================================================================')
    print('Working with:', dataset[d])
    print('===================================================================================================')

    for r in n_round:  # number of rounds loop
        comms_round = r
        for cl in n_clients:  # number of clients loop
            number_of_clients = cl

            print('---------------------------------------------')
            print('No. of Clients:', number_of_clients)
            print('No. of Rounds:', comms_round)
            print('---------------------------------------------')

            features = np.array(use_data.iloc[:, range(0, use_data.shape[1] - 1)])  # feature set
            labels = use_data.iloc[:, -1]  # labels --> B : Benign and S

            # Do feature scaling
            X = preprocessing.StandardScaler().fit(features).transform(features)

            # binarize the labels
            lb = LabelBinarizer()
            y = lb.fit_transform(labels)

            # split data into training and test set
            X_train, X_test, y_train, y_test = train_test_split(X,
                                                                y, shuffle=True,
                                                                test_size=0.2,
                                                                random_state=100)

            # create clients -- Horizontal FL
            clients = create_clients(X_train, y_train, num_clients=number_of_clients, initial='client')

            # process and batch the training data for each client
            clients_batched = dict()
            for (client_name, data) in clients.items():
                clients_batched[client_name] = batch_data(data)

            # process and batch the test set
            test_batched = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                                                                                     torch.tensor(y_test, dtype=torch.float32)),
                                                       batch_size=len(y_test), shuffle=False)

            # ==============================================
            # Traditional FedAvg 2017
            # ==============================================
            # -----------------------------------------------

            all_results = list()

            # create optimizer
            lr = 0.01
            loss = nn.BCELoss()
            optimizer = optim.SGD

            # initialize global model
            smlp_global = SimpleMLP(X.shape[1], 1)
            global_model = smlp_global
            # -----------------------------------------------

            print('|=======================|')
            print('|Traditional FedAvg 2017|')
            print('|=======================|')

            # commence global training loop
            for comm_round in range(comms_round):
                # get the global model's weights - will serve as the initial weights for all local models
                global_weights = [param.data.clone() for param in global_model.parameters()]
                print(global_weights)
                # initial list to collect local model weights after scaling
                scaled_local_weight_list = list()

                # randomize client data - using keys
                client_names = list(clients_batched.keys())
                random.shuffle(client_names)

                for client in client_names:
                    smlp_local = SimpleMLP(X.shape[1], 1)
                    local_model = smlp_local
                    # set local model weight to the weight of the global model
                    local_model.load_state_dict({name: param.clone() for name, param in zip(local_model.state_dict(), global_weights)})
                    optimizer = torch.optim.SGD(local_model.parameters(), lr=0.01)

                    # fit local model with client's data
                    train_loader = DataLoader(TensorDataset(torch.tensor(clients_batched[client].dataset.tensors[0], dtype=torch.float32),
                                                            torch.tensor(clients_batched[client].dataset.tensors[1], dtype=torch.float32)),
                                              batch_size=32, shuffle=True)

                    # List of training approaches
                    training_approaches = [train_model, fgsm_attack]

                    # Randomly choose between normal training and FGSM attack
                    selected_training_approach = random.choice(training_approaches)

                    # Apply the selected training approach
                    if selected_training_approach == train_model:
                        train_model(local_model, train_loader, loss, optimizer)
                    elif selected_training_approach == fgsm_attack:
                        epsilon = 1
                        X_adv = fgsm_attack(local_model, clients_batched[client].dataset.tensors[0], clients_batched[client].dataset.tensors[1], epsilon, loss, optimizer)
                        train_model(local_model, X_adv, loss, optimizer)
                    # train_model(local_model, train_loader, loss, optimizer)
                    # # FGSM attack
                    # epsilon = 1
                    # X_adv = fgsm_attack(local_model, clients_batched[client].dataset.tensors[0], clients_batched[client].dataset.tensors[1], epsilon,loss, optimizer)                  
                    # train_model(local_model, X_adv, loss, optimizer)

                    # scale the model weights and add to the list
                    scaling_factor = weight_scalling_factor(clients_batched, client)
                    scaled_weights = scale_model_weights(local_model.state_dict().values(), scaling_factor)
                    scaled_local_weight_list.append(scaled_weights)

                    # clear session to free memory after each communication round
                    torch.cuda.empty_cache()

                # to get the average over all the local model, we simply take the sum of the scaled weights
                average_weights = sum_scaled_weights(scaled_local_weight_list)

                # update global model
                for param, avg_param in zip(global_model.parameters(), average_weights):
                    param.data.copy_(avg_param)

                # test global model and print out metrics after each communications round
                for X_test_batch, Y_test_batch in test_batched:
                    global_acc, global_loss, global_f1, global_precision, global_recall, global_auc, global_fpr = test_model(X_test_batch, Y_test_batch, global_model, comm_round)
                    all_results.append([global_acc, global_loss, global_f1, global_precision, global_recall, global_auc, global_fpr])

            all_R = pd.DataFrame(all_results, columns=['global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
            flname = f'results/round-{r}/{cl}-clients/FedAvg-{dataset[d]}-results.csv'
            all_R.to_csv(flname, index=None)

            all_avg.append(np.concatenate(([dataset[d], r, cl], np.mean(all_results, axis=0))))  # Storing avg values for each dataset
            all_std.append(np.concatenate(([dataset[d], r, cl], np.std(all_results, axis=0))))  # Storing std values for each dataset

ALL_AVG = pd.DataFrame(all_avg, columns=['Dataset', 'num of round', 'num of clients', 'global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
ALL_AVG.to_csv('FedAvg-results.csv')

ALL_STD = pd.DataFrame(all_std, columns=['Dataset', 'num of round', 'num of clients', 'global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
ALL_STD.to_csv('FedAvg-all-std-results.csv')


Working with: Drebin
---------------------------------------------
No. of Clients: 5
No. of Rounds: 200
---------------------------------------------
|=======================|
|Traditional FedAvg 2017|
|=======================|
[tensor([[-0.0053,  0.0590,  0.0018,  ..., -0.0481,  0.0512,  0.0559],
        [-0.0221, -0.0615,  0.0247,  ..., -0.0115, -0.0480, -0.0197],
        [-0.0148, -0.0484, -0.0506,  ...,  0.0346, -0.0553, -0.0088],
        ...,
        [-0.0356,  0.0318,  0.0298,  ..., -0.0449,  0.0507,  0.0155],
        [-0.0256, -0.0443,  0.0537,  ...,  0.0578, -0.0459, -0.0309],
        [ 0.0149,  0.0130, -0.0198,  ..., -0.0031, -0.0446, -0.0598]]), tensor([ 0.0121,  0.0671,  0.0167,  0.0387,  0.0363, -0.0159, -0.0429,  0.0425,
        -0.0571,  0.0180, -0.0542,  0.0583, -0.0541,  0.0524,  0.0329, -0.0453,
        -0.0371,  0.0226,  0.0621, -0.0539,  0.0446, -0.0288,  0.0596, -0.0619,
         0.0555, -0.0239, -0.0026,  0.0362,  0.0493, -0.0556, -0.0438, -0.0463,
        -0.0673,

KeyboardInterrupt: 

In [ ]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import random



def fgsm_attack(model, X, y, epsilon,loss, optimizer):
    # Generate adversarial examples
    model.eval()
    X_adv = X.detach().clone()
    X_adv.requires_grad = True
    outputs = model(X_adv)
    loss = loss(outputs, y)
    # print("adver" , loss)
    loss.backward()
    grad_sign = X_adv.grad.data.sign()
    X_adv = X_adv + epsilon * grad_sign
    X_adv = torch.clamp(X_adv, 0, 1)
    train_loader = DataLoader(TensorDataset(torch.tensor(X_adv, dtype=torch.float32),
                                        torch.tensor(y, dtype=torch.float32)),
                          batch_size=32, shuffle=True)
    return train_loader

def train_model(model, train_loader, loss_fn, optimizer):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        # print("adver" , loss)

        loss.backward()
        optimizer.step()

all_avg = []
all_std = []

n_clients = [5, 10, 15]
n_round = [200, 5000]

dataset = ['Drebin', 'Malgenome', 'Kronodroid', 'Tuandromd']

# for d in range(0,len(dataset)):
for d in range(0, 2):
    if d == 0:
        use_data = Drebin_data
    elif d == 1:
        use_data = Malgenome_data
    elif d == 2:
        use_data = Kronodroid_data
    elif d == 3:
        use_data = Tuandromd_data

    print('===================================================================================================')
    print('Working with:', dataset[d])
    print('===================================================================================================')

    for r in n_round:  # number of rounds loop
        comms_round = r
        for cl in n_clients:  # number of clients loop
            number_of_clients = cl

            print('---------------------------------------------')
            print('No. of Clients:', number_of_clients)
            print('No. of Rounds:', comms_round)
            print('---------------------------------------------')

            features = np.array(use_data.iloc[:, range(0, use_data.shape[1] - 1)])  # feature set
            labels = use_data.iloc[:, -1]  # labels --> B : Benign and S

            # Do feature scaling
            X = preprocessing.StandardScaler().fit(features).transform(features)

            # binarize the labels
            lb = LabelBinarizer()
            y = lb.fit_transform(labels)

            # split data into training and test set
            X_train, X_test, y_train, y_test = train_test_split(X,
                                                                y, shuffle=True,
                                                                test_size=0.2,
                                                                random_state=100)

            # create clients -- Horizontal FL
            clients = create_clients(X_train, y_train, num_clients=number_of_clients, initial='client')

            # process and batch the training data for each client
            clients_batched = dict()
            for (client_name, data) in clients.items():
                clients_batched[client_name] = batch_data(data)

            # process and batch the test set
            test_batched = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                                                                                     torch.tensor(y_test, dtype=torch.float32)),
                                                       batch_size=len(y_test), shuffle=False)

            # ==============================================
            # Traditional FedAvg 2017
            # ==============================================
            # -----------------------------------------------

            all_results = list()

            # create optimizer
            lr = 0.01
            loss = nn.BCELoss()
            optimizer = optim.SGD

            # initialize global model
            smlp_global = SimpleMLP(X.shape[1], 1)
            global_model = smlp_global
            # -----------------------------------------------

            print('|=======================|')
            print('|Traditional FedAvg 2017|')
            print('|=======================|')

            # commence global training loop
            for comm_round in range(comms_round):
                # get the global model's weights - will serve as the initial weights for all local models
                global_weights = [param.data.clone() for param in global_model.parameters()]
                print(global_weights)
                # initial list to collect local model weights after scaling
                scaled_local_weight_list = list()

                # randomize client data - using keys
                client_names = list(clients_batched.keys())
                random.shuffle(client_names)

                for client in client_names:
                    smlp_local = SimpleMLP(X.shape[1], 1)
                    local_model = smlp_local
                    # set local model weight to the weight of the global model
                    local_model.load_state_dict({name: param.clone() for name, param in zip(local_model.state_dict(), global_weights)})
                    optimizer = torch.optim.SGD(local_model.parameters(), lr=0.01)

                    # fit local model with client's data
                    train_loader = DataLoader(TensorDataset(torch.tensor(clients_batched[client].dataset.tensors[0], dtype=torch.float32),
                                                            torch.tensor(clients_batched[client].dataset.tensors[1], dtype=torch.float32)),
                                              batch_size=32, shuffle=True)

                    # List of training approaches
                    training_approaches = [train_model, fgsm_attack]

                    # Randomly choose between normal training and FGSM attack
                    selected_training_approach = random.choice(training_approaches)

                    # Apply the selected training approach
                    if selected_training_approach == train_model:
                        train_model(local_model, train_loader, loss, optimizer)
                    elif selected_training_approach == fgsm_attack:
                        epsilon = 1
                        X_adv = fgsm_attack(local_model, clients_batched[client].dataset.tensors[0], clients_batched[client].dataset.tensors[1], epsilon, loss, optimizer)
                        train_model(local_model, X_adv, loss, optimizer)
                    # train_model(local_model, train_loader, loss, optimizer)
                    # # FGSM attack
                    # epsilon = 1
                    # X_adv = fgsm_attack(local_model, clients_batched[client].dataset.tensors[0], clients_batched[client].dataset.tensors[1], epsilon,loss, optimizer)                  
                    # train_model(local_model, X_adv, loss, optimizer)

                    # scale the model weights and add to the list
                    scaling_factor = weight_scalling_factor(clients_batched, client)
                    scaled_weights = scale_model_weights(local_model.state_dict().values(), scaling_factor)
                    scaled_local_weight_list.append(scaled_weights)

                    # clear session to free memory after each communication round
                    torch.cuda.empty_cache()

                # to get the average over all the local model, we simply take the sum of the scaled weights
                average_weights = sum_scaled_weights(scaled_local_weight_list)

                # update global model
                for param, avg_param in zip(global_model.parameters(), average_weights):
                    param.data.copy_(avg_param)

                # test global model and print out metrics after each communications round
                for X_test_batch, Y_test_batch in test_batched:
                    global_acc, global_loss, global_f1, global_precision, global_recall, global_auc, global_fpr = test_model(X_test_batch, Y_test_batch, global_model, comm_round)
                    all_results.append([global_acc, global_loss, global_f1, global_precision, global_recall, global_auc, global_fpr])

            all_R = pd.DataFrame(all_results, columns=['global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
            flname = f'results/round-{r}/{cl}-clients/FedAvg-{dataset[d]}-results.csv'
            all_R.to_csv(flname, index=None)

            all_avg.append(np.concatenate(([dataset[d], r, cl], np.mean(all_results, axis=0))))  # Storing avg values for each dataset
            all_std.append(np.concatenate(([dataset[d], r, cl], np.std(all_results, axis=0))))  # Storing std values for each dataset

ALL_AVG = pd.DataFrame(all_avg, columns=['Dataset', 'num of round', 'num of clients', 'global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
ALL_AVG.to_csv('FedAvg-results.csv')

ALL_STD = pd.DataFrame(all_std, columns=['Dataset', 'num of round', 'num of clients', 'global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
ALL_STD.to_csv('FedAvg-all-std-results.csv')
